In [87]:
import os, sys, time, datetime
import pandas as pd
import numpy as np

In [88]:
#importation des fichers de données
internet_data = pd.read_csv('data/internet_access_2017.csv')
density_data = pd.read_csv('data/population_density_2017.csv')
population_data = pd.read_csv('data/population_total_2017.csv')
superficie_data = pd.read_csv('data/superficie_2017.csv')

#importation du ficher metadata
metadata = pd.read_csv('data/metadata.csv')

In [89]:
internet_data.head(3)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
0,Aruba,ABW,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,62.0,69.0,74.000000,78.9,83.78,88.661227,93.542454,97.170000,NaN,NaN
1,Afghanistan,AFG,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,5.0,5.454545,5.9,7.00,8.260000,10.595726,11.447688,NaN,NaN
2,Angola,AGO,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,2.8,3.1,6.500000,8.9,10.20,12.400000,13.000000,14.339079,NaN,NaN


In [90]:
#suppression des colonnes inutiles sur internet_data
internet = internet_data.loc[ :,["Country Name", "Country Code", "2017" ]].rename(index=str, columns={"2017": "internet"})
internet['internet'] = internet["internet"].round(1)


#on degage tout ce dont on n'a pas besoin sur le dataframe density_data
density = density_data.loc[ :,["Country Name","2017" ]].round(2).rename(index=str, columns={"2017": "density"})

#on degage tout ce dont on n'a pas besoin sur le dataframe population_data
population = population_data.loc[ :,["Country Name","2017" ]].rename(index=str, columns={"2017": "population"})
population['population'] = population["population"].div(1000000).round(2)

#On degage tout ce dont on n'a pas besoin sur le dataframe superficie_data
superficie_latest = superficie_data.loc[ :,["Country Name", "2017" ]].rename(index=str, columns={"2017": "superficie"})
superficie_latest['superficie'] = superficie_latest["superficie"].div(1000).round(3)


#suppression des colonnes inutiles sur metadata
metadata = metadata.iloc[:,:-3]

#remplassement des valeurs NAN par Other dans les Categories Region et IncomeGroup
values = {'Region': 'Other', 'IncomeGroup': 'Other'}
metadata.loc[:,['Region', 'IncomeGroup']] = metadata.loc[:,['Region', 'IncomeGroup']].fillna(value=values)

In [91]:
#on commence par merger les trois datasets
merged = pd.merge(internet, density, how='left', on=['Country Name'])
merged = pd.merge(merged, population, how='left', on=['Country Name'])
merged = pd.merge(merged, superficie_latest, how='left', on=['Country Name'])

#puis on fusionne avec le dataset metadate
merged = pd.merge(merged, metadata, how='left', on=['Country Code'])

In [92]:
#on degage toutes les lignes invalides (internet, superficie, population = 0)
merged = merged.loc[merged['internet'] > 0,:].loc[merged['superficie'] > 0,:].loc[merged['population'] > 0,:]

In [93]:
#on met en forme notre tableaux
pivot_internet = pd.pivot_table(merged, values=['population', 'superficie', 'density', 'internet'],index=['Region', 'Country Name', 'IncomeGroup'],
                            fill_value=0)


In [94]:
#on créer des tableaux pour visualiser nos données
stats_internet = pivot_internet[['population', 'superficie', 'density', 'internet']].sort_values(['Region','internet'], ascending=[1,0])
stats_superficie = pivot_internet[['population', 'density', 'superficie', 'internet']].sort_values(['Region','superficie'], ascending=[1,0])
stats_density = pivot_internet[['population', 'superficie', 'density', 'internet']].sort_values(['Region','density'], ascending=[1,0])
stats_population = pivot_internet[['superficie', 'density', 'population', 'internet']].sort_values(['Region','population'], ascending=[1,1])


In [95]:
#Example: On récupère les 20 pays Européens classés par accès à internet en 2017
stats_internet.loc['Sub-Saharan Africa'].head(10)

,,population,superficie,density,internet
Country Name,IncomeGroup,,,,
Seychelles,High income,0.10,0.46,208.35,58.8
Cabo Verde,Lower middle income,0.55,4.03,135.58,57.2
South Africa,Upper middle income,56.72,1213.09,46.75,56.2
Mauritius,Upper middle income,1.26,2.03,622.96,55.6
Gabon,Upper middle income,2.03,257.67,7.86,50.3
Cote d'Ivoire,Lower middle income,24.29,318.00,76.40,43.8
Botswana,Upper middle income,2.29,566.73,4.04,41.4
Ghana,Lower middle income,28.83,227.54,126.72,37.9
Namibia,Upper middle income,2.53,823.29,3.08,36.8


In [96]:
#on peut aussi ranger tous nos tableaux dans un dict
stats = {}
stats['internet'] = pivot_internet[['population', 'superficie', 'density', 'internet']].sort_values(['Region','internet'], ascending=[1,0])
stats['superficie'] = pivot_internet[['population', 'density', 'superficie', 'internet']].sort_values(['Region','superficie'], ascending=[1,0])
stats['density'] = pivot_internet[['population', 'superficie', 'density', 'internet']].sort_values(['Region','density'], ascending=[1,0])
stats['population'] = pivot_internet[['superficie', 'density', 'population', 'internet']].sort_values(['Region','population'], ascending=[1,1])


In [97]:
#Example: On récupère les 20 pays Européens classés par densité de population en 2017
stats['density'].loc['Europe & Central Asia'].head(10)

,,population,superficie,density,internet
Country Name,IncomeGroup,,,,
Monaco,High income,0.04,0.002,19347.50,97.1
San Marino,High income,0.03,0.060,556.67,60.2
Netherlands,High income,17.13,33.690,508.50,93.2
Belgium,High income,11.38,30.280,375.90,87.7
United Kingdom,High income,66.02,241.930,272.90,94.6
Luxembourg,High income,0.60,2.430,245.41,97.8
Liechtenstein,High income,0.04,0.160,237.01,98.1
Germany,High income,82.69,349.360,236.68,84.4
Switzerland,High income,8.45,39.516,213.86,93.7
